In [ ]:
# ADZ ADDED THIS TO MAKE MODULE IMPORTING WORK IN JUPYTER NOTEBOOK
# NOTE: WHY DO I HAVE TO RUN 3rd CELL TWICE TO GET IT TO RUN? GET ERROR FIRST TIME...

In [ ]:
!pip install import-ipynb

In [1]:
#FROM CAMMERONN GITHUB smemp.py
"""
@filename smemp.py

Derive properties of target stars using the Empirical SpecMatch algorithm and
write them to a csv file.
Display plots and figures of the process (optional).
"""
import import_ipynb #ADZ ADD 6/23/20
import sys, os
from os import listdir
from os.path import isfile, join
import csv
from pylab import *
import pylab
import astropy.io.fits as pf
sys.path.append('specmatch-emp')
import specmatchemp.library
import specmatchemp.plots as smplot
from specmatchemp.spectrum import Spectrum
from specmatchemp.specmatch import SpecMatch
from deblaze import afs_deblaze
from rescale import get_rescaled_wave_soln
from rescale import resample
from optparse import OptionParser
from testing_deblaze import normalized_registered_and_deblazed_array #ADZ ADD 6/23/20 (this is Jackie Telson's function)


importing Jupyter notebook from deblaze.ipynb
importing Jupyter notebook from rescale.ipynb
importing Jupyter notebook from testing_deblaze.ipynb


In [2]:
def write_results(fd, my_spectrum, sm, write_new = False):
    """
    Write to a csv the derived properties of a target following
    the SpecMatch process
    Args:
        fd (File): object for the csv file to write to
        my_spectrum (spectrum.Spectrum): Target spectrum
        sm (specmatch.SpecMatch): Contains results of the algorithm
        write_new (Boolean): Whether to write to a new csv file
    """

    fieldnames = ['apf_name', 'Teff', 'u_Teff','radius','u_radius',
                  'logg','u_logg', 'feh','u_feh','mass','u_mass','age',
                  'u_age', 'best_mean_chi_squared']

    thewriter = csv.DictWriter(fd, fieldnames=fieldnames)

    if (write_new): thewriter.writeheader()

    thewriter.writerow({'apf_name' : my_spectrum.name,
                        'Teff' : '{0:.3f}'.format(sm.results['Teff']),
                        'u_Teff' : '{0:.3f}'.format(sm.results['u_Teff']),
                        'radius' : '{0:.3f}'.format(sm.results['radius']),
                        'u_radius' : '{0:.3f}'.format(sm.results['u_radius']),
                        'logg' : '{0:.3f}'.format(sm.results['logg']),
                        'u_logg' : '{0:.3f}'.format(sm.results['u_logg']),
                        'feh' : '{0:.3f}'.format(sm.results['feh']),
                        'u_feh' : '{0:.2f}'.format(sm.results['u_feh']),
                        'mass' : '{0:.3f}'.format(sm.results['mass']),
                        'u_mass' : '{0:.3f}'.format(sm.results['u_mass']),
                        'age' : '{0:.2f}'.format(sm.results['age']),
                        'u_age' : '{0:.2f}'.format(sm.results['u_age']),
                        'best_mean_chi_squared' :
                        '{0:.2f}'.format(best_mean_chi_squared)})

parser = OptionParser()
parser.add_option("-f", "--file", action='store', type='string',
                  dest="pathname",
                  help="pass the path of the FITS file(s) as an argument")
parser.add_option("-o", action='store', type='string',
                  dest="outputpath",
                  help="pass the path to a csv file to write to "
                       "as an argument")
parser.add_option("-p", action="store_true", dest="plot",
                  help='plot')
parser.add_option("--all", action="store_true", dest="all",
                  help='plots all wavelength regions')
parser.add_option("--best", action="store_true", dest="best",
                  help='plots the reference, modified reference and residuals '
                       'for each of the best matches.')
parser.add_option("--chi", action="store_true", dest="chi",
                  help='plot the chi-squared surface from the pairwise \
                  matching procedure')
parser.add_option("--ref", action="store_true", dest="ref",
                  help='plot the locations of the best references used in the \
                  linear combination step')
parser.add_option("--sr", action="store_true", dest="ref",
                  help='save the residuals ')

(options, sys.argv) = parser.parse_args(sys.argv)

print('EMPIRICAL SPECMATCH'), print()

# if no path given in command, prompt user for a path to a file or a directory
# from which to acquire fits files
# NOTE: program currently only known to work if all the files in the directory
# are fits files and are intended targets

#del pathname #ADZ REMOVE IF AND ELSE othewise, on the first run after restarting kernel it thinks 
# options.pathname is not None and so never asks for file
#if (options.pathname == None):
path_name = input('Please enter the path to the FITS file(s) of a star: ')
print()
#else:
#    path_name = options.pathname
    
try:
    filenames = [f for f in listdir(path_name) if isfile(join(path_name, f))]
except NotADirectoryError: # path to one file
    path_split = path_name.split('/')
    path_split[:-1] = ['/'.join(path_split[:-1])]
    filenames = []
    filename = path_split[-1]
    filenames.append(filename)
    path_name = path_split[0]

# check to see if files are for the same star
# NOTE: program currently does not work if the input directory contains fits
# files for multiple stars
names = set()

#ADZ ADD FOR DEBUGGING 6/23/
#try:
#    print(pathname)
#except NameError:
#    print('nope, no pathname yet')
#for filename in filenames:
#    print('I think the filename is: ' + filename)

for filename in filenames:
    print(filename) #ADZ ADD FOR DEBUGGING
    file = pf.open(path_name + '/' + filename,ignore_missing_end=True) #ADZ ADD ignore_missing_end=True
    header = file[0].header
    name = header['TOBJECT']
    names.add(name)
    if (len(names) > 1):
        print('Spectra Addition Error: ')
        print('This program sums the spectra for a star.')
        print('Please only provide the path to FITS files for the same star' +
              ' for a run of this program.')
        sys.exit()

display_plots = False
if (options.plot or options.chi or options.best or options.ref):
    display_plots = True # bool var for whether or not to display plots

# Prompt for regions to plot
if ((display_plots) and (options.all == None)):
    print("0 : 5000 to 5100 Å")
    print("1 : 5100 to 5200 Å")
    print("2 : 5200 to 5300 Å")
    print("3 : 5300 to 5400 Å")
    print("4 : 5400 to 5500 Å")
    print("5 : 5500 to 5600 Å")
    print("6 : 5600 to 5700 Å")
    print("7 : 5700 to 5800 Å")
    print("8 : 5800 to 5900 Å")
    print("Please enter the corresponding numbers for " +
          "the wavelength regions to be plotted.")
    print("Separate the numbers with spaces.")
    print("Default option is only region 1. Simply press enter for " +
          "default option.")
    print("Enter \'all\' to plot all the regions.")

    while(True):
        inp = input('→ ')
        try:
            if (inp == ''): # Default - plot region 1
                regions = [1]
            elif (inp == 'all'):
                regions = (list(range(9)))
            else:
                regions = [int(region) for region in sort(inp.split(" "))]
                if (False in [(0 <= region <= 8) for region in regions]):
                    continue
        except ValueError: continue
        break

else: # plot all
    regions = (list(range(9)))

# Read in data from wavelength solution
wave_soln = (pf.open('apf_wav.fits'))[0].data

# Sum all of the data files for a star
data = np.zeros((79, 4608))

ve = False
for filename in filenames:
    file = pf.open(path_name + '/' + filename)
    data_part = file[0].data
    if (str(np.shape(data_part)) != '(79, 4608)'):
        print(str(np.shape(data_part)) + ' is an incompatible data shape.')
        print('Cannot perform shift-and-match process.')
        sys.exit()
    try:
        data += data_part
    except ValueError:
        ve = True

if (ve):
    print("Value Error occurred during spectra summation.")

header = file[0].header
name = header['TOBJECT']
print('Running SpecMatch-Emp on ' + name + ':')
for filename in filenames:
    print(filename)

ve = False
#Deblaze the orders: 31 to 52 (range of focus in the SME library)
for order_inc in range(22):
    try: #ADZ CHANGE THIS TRY STATEMENT 6/23/20
       # data[30 + order_inc, :4607] = afs_deblaze(data[30 + order_inc],
       #                                           30 + order_inc)[0]
        #print(data)
        data[30 + order_inc, :4600] = normalized_registered_and_deblazed_array(file,30 + order_inc)
        print(order_inc)
    except ValueError: ve = True

if (ve): print("Value Error occurred during blaze correction.")

print('got here line 180')   

# Get a wavelength solution rescaled onto the scale of the library
wave_soln_ref = get_rescaled_wave_soln()
print('got here line 184')
# Resample the spectrum onto the new wavelength scale
data_new = resample(wave_soln_ref, wave_soln, data)
print('got here line 187')
# Create spectrum object
my_spectrum = Spectrum(np.asarray(wave_soln_ref), np.asarray(data_new))
my_spectrum.name = name
print('got here line 191')
lib = specmatchemp.library.read_hdf()

sm = SpecMatch(my_spectrum, lib)

# Perform shift
sm.shift()

# Perform match
sm.match()

# Perform lincomb
sm.lincomb()

print('got here line 205') #ADZ DELETE THIS!

# Chi squared values of the best match
chi_squares = []

chi_squares.append(sm.match_results.iloc[0]['chi_squared_5000'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5100'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5200'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5300'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5400'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5500'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5600'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5700'])
chi_squares.append(sm.match_results.iloc[0]['chi_squared_5800'])

best_mean_chi_squared = np.mean(np.asarray(chi_squares))

# Record results to csv file
# if not provided a output file name, temp.csv is written to
# (and created if not already)
if (options.outputpath != None):
    if (not options.outputpath.endswith('.csv')):
        if (not options.outputpath.endswith('/')):
            options.outputpath += '/'
        options.outputpath += 'temp.csv'
    if isfile(options.outputpath):
        with open(options.outputpath,'a') as fd:
            write_results(fd, my_spectrum, sm)
    else:
        with open(options.outputpath, 'w', newline='') as fd:
            write_results(fd, my_spectrum, sm, write_new = True)
else:
    if isfile('temp.csv'):
        with open('temp.csv','a') as fd: write_results(fd, my_spectrum, sm)
    else:
        with open('temp.csv', 'w', newline='') as fd:
            write_results(fd, my_spectrum, sm, write_new = True)

print('got here 243') #ADZ DELETE THIS!

if (display_plots):

    # Plot figures
    fignum = 0
    for r in regions:

        if (options.chi):
            fignum += 1
            plt.figure(fignum)
            fig3 = pylab.gcf()
            fig3.canvas.set_window_title('Chi-Squared Surface: Region ' + str(r))
            sm.plot_chi_squared_surface(region=r, num_best=None)

        if (options.best):
            fignum += 1
            plt.figure(fignum)
            fig1 = pylab.gcf()
            fig1.canvas.set_window_title('Best Match Spectra: Region ' + str(r))
            sm.plot_best_match_spectra(region=r, wavlim='all', num_best=None)

        if (options.ref):
            fignum += 1
            plt.figure(fignum)
            fig2 = pylab.gcf()
            fig2.canvas.set_window_title('References: Region ' + str(r))
            sm.plot_references(region=r, num_best=None, verbose=True)

        fignum += 1
        plt.figure(fignum)
        fig = pylab.gcf()
        fig.canvas.set_window_title('Linear Combination: Region ' + str(r))
        sm.plot_lincomb(region=r, wavlim='all')

        plt.show()

    # Plot shift
    fignum += 1
    plt.figure(fignum, figsize=(10,5))
    fig4 = pylab.gcf()
    fig4.canvas.set_window_title('Shift')
    sm.target_unshifted.plot(normalize=True, plt_kw={'color':'forestgreen'},
                             text='Target (unshifted)')
    sm.target.plot(offset=0.5, plt_kw={'color':'royalblue'},
                   text='Target: ' + name + ' (shifted)')
    sm.shift_ref.plot(offset=1, plt_kw={'color':'firebrick'},
                      text='Reference: ' +sm.shift_ref.name)
    plt.xlim(5160,5190)
    plt.ylim(0,2.2)

    plt.show()

print('COMPLETE')


EMPIRICAL SPECMATCH



Please enter the path to the FITS file(s) of a star:  ./raou.267.fits



nope, no pathname yet
I think the filename is: raou.267.fits
raou.267.fits
Running SpecMatch-Emp on HIP73695:
raou.267.fits
got through deblaze function
0
got through deblaze function
1
got through deblaze function
2
got through deblaze function
3
got through deblaze function
4
got through deblaze function
5
got through deblaze function
6
got through deblaze function
7
got through deblaze function
8
got through deblaze function
9
got through deblaze function
10
got through deblaze function
11
got through deblaze function
12
got through deblaze function
13
got through deblaze function
14
got through deblaze function
15
got through deblaze function
16
got through deblaze function
17
got through deblaze function
18
got through deblaze function
19
got through deblaze function
20
got through deblaze function
21
got here line 180
got into rescale function
got here rescale 1
got here rescale 2
got through rescale
got here line 184
got here resample 1
got here resample 2
got here resample 3
G

In [ ]:
filename


In [ ]:
options.pathname

In [ ]:
#!wget https://github.com/stevecroft/bl-interns/tree/master/cameronn/afs_files
    

In [ ]:
#!wget -r -np -R "index.html*" https://github.com/stevecroft/bl-interns/tree/master/cameronn/afs_files
    

In [ ]:
#!wget https://raw.githubusercontent.com/stevecroft/bl-interns/master/cameronn/afs_files
    

In [3]:
print('oops')


oops


In [ ]:
sys.stdout = old_